<a href="https://www.kaggle.com/code/kaustubhratna/binary-aptos-resnet?scriptVersionId=120898965" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import cv2
import os
from zipfile import ZipFile
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.utils import img_to_array
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential

In [ ]:
dataset = []
labels = []

In [ ]:
def prepare_Images(label,path):
    img=cv2.imread(path,cv2.IMREAD_COLOR)
    try:
      img_res=cv2.resize(img,(256,256))
      img_array = img_to_array(img_res)
      img_array = img_array/255.0
      dataset.append(img_array)
      if str(label) == '0':
        labels.append('0')
      else:
        labels.append('1')
    except:
      print("error")

In [ ]:
train_Data = pd.read_csv("/kaggle/input/aptos2019-blindness-detection/train.csv")
train_Data.head()

In [ ]:
id_code_Data = train_Data['id_code']
diagnosis_Data = train_Data['diagnosis']

In [ ]:
for id_code,diagnosis in tqdm(zip(id_code_Data,diagnosis_Data)):
    path = os.path.join('/kaggle/input/aptos2019-blindness-detection/train_images','{}.png'.format(id_code))
    prepare_Images(diagnosis,path)

In [ ]:
images = np.array(dataset)
label_arr = np.array(labels)


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(images,label_arr,stratify=label_arr,test_size=0.20,random_state=44)


In [ ]:
y_train= np_utils.to_categorical(y_train, num_classes=2)
y_test = np_utils.to_categorical(y_test, num_classes=2)

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2
from sklearn.utils import class_weight, shuffle
from keras.losses import binary_crossentropy
from keras.applications.resnet import preprocess_input
import keras.backend as K
import tensorflow as tf
from sklearn.metrics import f1_score, fbeta_score

from sklearn.model_selection import train_test_split

input_shape = (128,256, 256, 3)
n_classes = 2

resnet_model = tf.keras.applications.ResNet50(
    input_shape=(256,256,3), 
    include_top=False, 
    weights='imagenet'
)

resnet_model.trainable =False

resnet_model.summary()


In [ ]:
resnettrain = tf.keras.Sequential([
    resnet_model,
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(2, activation='sigmoid')
])

resnettrain.summary()

In [ ]:
resnettrain.compile(
    optimizer="adam",
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

In [ ]:
history = resnettrain.fit(
    x_train,
    y_train,
    batch_size=128,
    verbose=1,
    epochs=100
    
)

In [ ]:
pred = resnettrain.predict(x_test)

resnettrain.evaluate(x_test,y_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
conMat = confusion_matrix(y_test.argmax(axis=1),pred.argmax(axis=1))
print(conMat)

In [ ]:

score = round(accuracy_score(y_test.argmax(axis=1), pred.argmax(axis=1)),2)
print(score)

In [ ]:
from sklearn.preprocessing import normalize
import seaborn as sns
normed_confusion_matrix = normalize(conMat , axis = 1, norm = 'l1')
cm_df = pd.DataFrame(normed_confusion_matrix)
sns.heatmap (cm_df , annot =True)

In [ ]:
report = classification_report(y_test.argmax(axis=1), pred.argmax(axis=1))
print(report)

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']

plt.figure(figsize = (8,8))
plt.subplot(1,2,1)
plt.plot(range(100),acc,label='Training Accuracy')
plt.legend(loc="lower right")
plt.title("Training over 100 epochs")


In [ ]:
plt.figure(figsize = (8,8))
plt.subplot(1,2,1)
plt.plot(range(100),loss,label='Training Loss')
plt.legend(loc="upper right")
plt.title("Training Loss over 100 epochs")